# Live plotting of data

In this notebook we will demonstrate how to view data live through the web interface.

First, the necessary imports

In [ ]:
import os
from time import sleep

import numpy as np

import qcodes as qc
from qcodes.dataset import (
    Measurement,
    initialise_or_create_database_at,
    load_or_create_experiment,
)
from qcodes.instrument_drivers.mock_instruments import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)

import panel_gui
import IPython.lib.backgroundjobs as bg

# Mock set-up

We'll use simulated instruments from qcodes to make a mock setup

In [ ]:
dac = DummyInstrument("dac", gates=["ch1", "ch2"])
dmm = DummyInstrumentWithMeasurement(name="dmm", setter_instr=dac)

station = qc.Station(dmm, dac)

# Initialize or create the database
Let's initialize the database using qcodes or create one if it doesn't already exist
We'll also register the experiment and sample names which will be viewable from the plotting interface

In [ ]:
db_file_path = os.path.join(os.getcwd(), "live_plotting_tutorial.db")
initialise_or_create_database_at(db_file_path)
exp = load_or_create_experiment(
    experiment_name="live_plotting",
    sample_name="no sample",
)

# Launch the plotting interface in the background

Running this cell should open a new tab in your default browser with the plotting interface.
If it does not open you can click the link instead
To begin with, there will be no data in the database so nothing will be plotted

In [ ]:
jobs = bg.BackgroundJobManager()
jobs.new(panel_gui.serve_app, 'live_plotting_tutorial.db')

# Generate some data and watch it plot live
On the web interface, make sure to click 'Refresh'. This will set the plotting interface to periodically check for new data and plot it (every 3 seconds).
Then, run the following cell to run a fake experiment using one of the qcodes dummy instruments.
You should see the data automatically plotted in the web interface

In [ ]:
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))

meas.write_period = 0.1

with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v), (dmm.v1, get_v))
        sleep(1)

    dataset = datasaver.dataset



# Live plotting of 2D data
The next cell simulates an experiment in which two independent variables are changed and one dependent variable is measured.
The plotter automatically plots data as a colourmap when it detects that there are more than one
independent variable.
Try this out by running the cell. You may have to change the run id in the plotting interface.
By playing with 'Slice along' and 'Slice value' in the web interface, it's also possible to plot 1D cuts for particular values of the x or y axis.

In [ ]:
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,dac.ch2))

meas.write_period = 0.1

with meas.run() as datasaver:
    for set_v1 in np.linspace(0, 25, 251):
        dac.ch1.set(set_v1)
        for set_v2 in np.linspace(0, 25, 251):
            dac.ch2.set(set_v2)
            get_v = dmm.v1.get()
            datasaver.add_result((dac.ch1, set_v1), (dac.ch2, set_v2), (dmm.v1, get_v))
            dataset = datasaver.dataset
            sleep(0.05)


